In [ ]:
pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import json
import pandas as pd

In [ ]:
# FIND ALL THE MATCH NUMBER
url = "https://bwfworldchampionships.bwfbadminton.com/results/3996/totalenergies-bwf-world-championships-2021/2021-12-12/"
result = requests.get(url)
doc = BeautifulSoup(result.text, "html.parser")
body = doc.body
tags = body.find_all('li', {"class": re.compile("^stats stats-")})
tagsstr = str(tags)
class_name = re.findall(r'class="stats stats-[0-9]*', tagsstr)
class_numb = list()
numb = ""
for m in class_name:
  for i in m:
    if i.isdigit():
      numb = numb + i
  class_numb.append(int(numb))
  numb = ""

class_numb

[543,
 160,
 541,
 420,
 33,
 422,
 34,
 549,
 38,
 418,
 301,
 42,
 46,
 41,
 162,
 40,
 555,
 168,
 47,
 545,
 174,
 291,
 424,
 416,
 299,
 293,
 287,
 43,
 553,
 547,
 170,
 39,
 426,
 166,
 289,
 45,
 295,
 172,
 179]

In [ ]:
# GAME SCORE TAB
url = "https://bwfworldchampionships.bwfbadminton.com/results/3996/totalenergies-bwf-world-championships-2021/2021-12-15/?match=31&stab=game"
result = requests.get(url)
doc = BeautifulSoup(result.text, "html.parser")
body = doc.body
st = body.find("script").string
pattern = re.search(r'chartData = (.*?);.*?', st)
score = list()
if pattern is not None:
    data = json.loads(pattern.groups()[0])
    for i in range(0,len(data)):
      score.append(str(str(data[i].get("team1")) + "-" + str(data[i].get("team2"))))

score

['0-0',
 '1-0',
 '2-0',
 '3-0',
 '4-0',
 '5-0',
 '6-0',
 '6-1',
 '7-1',
 '8-1',
 '9-1',
 '10-1',
 '11-1',
 '12-1',
 '12-2',
 '13-2',
 '13-3',
 '14-3',
 '15-3',
 '16-3',
 '17-3',
 '18-3',
 '19-3',
 '19-4',
 '20-4',
 '21-4']

In [ ]:
# RESULT TAB
url = "https://bwfworldchampionships.bwfbadminton.com/results/3996/totalenergies-bwf-world-championships-2021/2021-12-12/?match=33&stab=result"
result = requests.get(url)
soup = BeautifulSoup(result.text, "html.parser")
doc = soup.find("section", {"class" : "container-livescore"})
doc1 = soup.find("li", {"class" : re.compile("draw-[A-Z]+ match-[0-9]+ active")})
player1_team1_name = doc.find_all("div", {"class": "player1-name"})[0].get_text().replace("\n", "")
try:
  player2_team1_name = doc.find_all("div", {"class": "player1-name"})[1].get_text().replace("\n", "")
except:
  player2_team1_name = "-"
player1_team2_name = doc.find_all("div", {"class": "player2-name"})[0].get_text().replace("\n", "")
try:
  player2_team2_name = doc.find_all("div", {"class": "player2-name"})[1].get_text().replace("\n", "")
except:
  player2_team2_name = "-"
team1_flag = doc.find("div", {"class": "player1-flag"}).find('img').get('title')
team2_flag = doc.find("div", {"class": "player2-flag"}).find('img').get('title')
round = doc1.find("div", {"class": "round"}).get_text().replace("\n", "")
check = doc1.find("div", {"class" : "score"}).get_text().replace("\n", "")
try:
  check1 = re.search("[A-Za-z]+", check)
except:
  check1 = None

if check1 is None:
  walkover = "FALSE"
  retired = "FALSE"
  game1_score = doc.find_all("div", {"class": re.compile("^game-score-1")})[0].get_text().replace("\n", "").replace(" ", "") + "-" + doc.find_all("div", {"class": re.compile("^game-score-2")})[0].get_text().replace("\n", "").replace(" ", "")
  game2_score = doc.find_all("div", {"class": re.compile("^game-score-1")})[1].get_text().replace("\n", "").replace(" ", "") + "-" + doc.find_all("div", {"class": re.compile("^game-score-2")})[1].get_text().replace("\n", "").replace(" ", "")
  try:
    game3_score = doc.find_all("div", {"class": re.compile("^game-score-1")})[2].get_text().replace("\n", "").replace(" ", "") + "-" + doc.find_all("div", {"class": re.compile("^game-score-2")})[2].get_text().replace("\n", "").replace(" ", "")
  except:
    game3_score = ""
  if doc.find_all("div", {"class": re.compile("^game-score-1")})[0].get_text().replace("\n", "").replace(" ", "") > doc.find_all("div", {"class": re.compile("^game-score-2")})[0].get_text().replace("\n", "").replace(" ", ""):
    game1_win = 1
  else:
    game1_win = 2
    
  if doc.find_all("div", {"class": re.compile("^game-score-1")})[1].get_text().replace("\n", "").replace(" ", "") > doc.find_all("div", {"class": re.compile("^game-score-2")})[1].get_text().replace("\n", "").replace(" ", ""):
    game2_win = 1
  else:
    game2_win = 2
  
  if doc.find_all("div", {"class": re.compile("^game-score-1")})[2].get_text().replace("\n", "").replace(" ", "") > doc.find_all("div", {"class": re.compile("^game-score-2")})[2].get_text().replace("\n", "").replace(" ", ""):
    game3_win = 1
  else:
    game3_win = 2

else:
  if check1.group() == "Walkover":
    walkover = "TRUE"
    retired = "FALSE"
    game1_score = "-"
    game2_score = "-"
    game3_score = "-"
    game1_win = 1
    game2_win = 1
    game3_win = "-"
  if check1.group() == "Retired":
    walkover = "FALSE"
    retired = "TRUE"
    game1_score = doc.find_all("div", {"class": re.compile("^game-score-1")})[0].get_text().replace("\n", "").replace(" ", "") + "-" + doc.find_all("div", {"class": re.compile("^game-score-2")})[0].get_text().replace("\n", "").replace(" ", "")
    game2_score = "-"
    game3_score = "-"
    if doc.find_all("div", {"class": re.compile("^game-score-1")})[0].get_text().replace("\n", "").replace(" ", "") > doc.find_all("div", {"class": re.compile("^game-score-2")})[0].get_text().replace("\n", "").replace(" ", ""):
      game1_win = 1
    else:
      game1_win = 2 
      game2_win = game1_win
      game3_win = "-"


In [ ]:
player1_team1_name, player2_team1_name, player1_team2_name, player2_team2_name, team1_flag, team2_flag, game1_score, game2_score, game3_score, game1_win, game2_win, game3_win, round, walkover, retired

('Kevin CORDON ',
 '-',
 'Joran KWEEKEL ',
 '-',
 'GUA',
 'NED',
 '21-18',
 '10-21',
 '21-16',
 1,
 2,
 1,
 'MS',
 'FALSE',
 'FALSE')

In [ ]:
# H2H TAB
url = "https://bwfworldchampionships.bwfbadminton.com/results/3996/totalenergies-bwf-world-championships-2021/2021-12-12/?match=543&stab=h2h"
result = requests.get(url)
doc = BeautifulSoup(result.text, "html.parser")
h2h = doc.find("div", {"class": "cifr1"}).get_text().replace("\n", "").replace(" ", "") + "-" + doc.find("div", {"class": "cifr2"}).get_text().replace("\n", "").replace(" ", "")
h2h

'1-0'

In [ ]:
# RANKING TAB
url = "https://bwfworldchampionships.bwfbadminton.com/results/3996/totalenergies-bwf-world-championships-2021/2021-12-12/?match=31&stab=ranking"
result = requests.get(url)
doc = BeautifulSoup(result.text, "html.parser")

crank = doc.find("div", {"class": "cifr1"}).get_text().replace("\t", "").replace("\n", "").replace(" ", "") + "-" + doc.find("div", {"class": "cifr2"}).get_text().replace("\t", "").replace("\n", "").replace(" ", "")
crank

In [ ]:
# SCRAPING
#"2021-12-15", "2021-12-16", "2021-12-17", "2021-12-18", "2021-12-19"
dates = ["2021-12-12", "2021-12-13", "2021-12-14", "2021-12-15", "2021-12-16", "2021-12-17", "2021-12-18", "2021-12-19"]
rows = [] 
for date in dates:
  url = "https://bwfworldchampionships.bwfbadminton.com/results/3996/totalenergies-bwf-world-championships-2021/" + date + "/"
  result = requests.get(url)
  doc = BeautifulSoup(result.text, "html.parser")
  body = doc.body
  tags = body.find_all('li', {"class": re.compile("^stats stats-")})
  tagsstr = str(tags)
  class_name = re.findall(r'class="stats stats-[0-9]*', tagsstr)
  class_numb = list()
  numb = ""
  for m in class_name:
    for i in m:
      if i.isdigit():
        numb = numb + i
    class_numb.append(int(numb))
    numb = ""
  
  for number in class_numb:
    
    # RESULT TAB
    url = "https://bwfworldchampionships.bwfbadminton.com/results/3996/totalenergies-bwf-world-championships-2021/" + date + "/" + "?match=" + str(number) + "&stab=result"
    result = requests.get(url)
    soup = BeautifulSoup(result.text, "html.parser")
    doc = soup.find("section", {"class" : "container-livescore"})
    doc1 = soup.find("li", {"class" : re.compile("draw-[A-Z]+ match-[0-9]+ active")})
    
    player1_team1_name = doc.find_all("div", {"class": "player1-name"})[0].get_text().replace("\n", "")
    try:
      player2_team1_name = doc.find_all("div", {"class": "player1-name"})[1].get_text().replace("\n", "")
    except:
      player2_team1_name = "-"
    
    player1_team2_name = doc.find_all("div", {"class": "player2-name"})[0].get_text().replace("\n", "")
    try:
      player2_team2_name = doc.find_all("div", {"class": "player2-name"})[1].get_text().replace("\n", "")
    except:
      player2_team2_name = "-"
    
    team1_flag = doc.find("div", {"class": "player1-flag"}).find('img').get('title')
    team2_flag = doc.find("div", {"class": "player2-flag"}).find('img').get('title')
    round = doc1.find("div", {"class": "round"}).get_text().replace("\n", "")
    check = doc1.find("div", {"class" : "score"}).get_text().replace("\n", "")
    
    try:
      check1 = re.search("[A-Za-z]+", check)
    except:
      check1 = None

    if check1 is None:
      walkover = "FALSE"
      retired = "FALSE"
      
      game1_score = doc.find_all("div", {"class": re.compile("^game-score-1")})[0].get_text().replace("\n", "").replace(" ", "") + "-" + doc.find_all("div", {"class": re.compile("^game-score-2")})[0].get_text().replace("\n", "").replace(" ", "")
      game2_score = doc.find_all("div", {"class": re.compile("^game-score-1")})[1].get_text().replace("\n", "").replace(" ", "") + "-" + doc.find_all("div", {"class": re.compile("^game-score-2")})[1].get_text().replace("\n", "").replace(" ", "")
      
      try:
        game3_score = doc.find_all("div", {"class": re.compile("^game-score-1")})[2].get_text().replace("\n", "").replace(" ", "") + "-" + doc.find_all("div", {"class": re.compile("^game-score-2")})[2].get_text().replace("\n", "").replace(" ", "")
      except:
        game3_score = ""
      
      try:
        game3_score = doc.find_all("div", {"class": re.compile("^game-score-1")})[2].get_text().replace("\n", "").replace(" ", "") + "-" + doc.find_all("div", {"class": re.compile("^game-score-2")})[2].get_text().replace("\n", "").replace(" ", "")
      except:
        game3_score = ""
      
      if doc.find_all("div", {"class": re.compile("^game-score-1")})[0].get_text().replace("\n", "").replace(" ", "") > doc.find_all("div", {"class": re.compile("^game-score-2")})[0].get_text().replace("\n", "").replace(" ", ""):
        game1_win = 1
      else:
        game1_win = 2
      
      if doc.find_all("div", {"class": re.compile("^game-score-1")})[1].get_text().replace("\n", "").replace(" ", "") > doc.find_all("div", {"class": re.compile("^game-score-2")})[1].get_text().replace("\n", "").replace(" ", ""):
        game2_win = 1
      else:
        game2_win = 2
      
      if game3_score != "":
        if doc.find_all("div", {"class": re.compile("^game-score-1")})[2].get_text().replace("\n", "").replace(" ", "") > doc.find_all("div", {"class": re.compile("^game-score-2")})[2].get_text().replace("\n", "").replace(" ", ""):
          game3_win = 1
        else:
          game3_win = 2
      else:
        game3_win = ""
      
    else:
      if check1.group() == "Walkover":
        walkover = "TRUE"
        retired = "FALSE"
        game1_score = "-"
        game2_score = "-"
        game3_score = "-"
        game1_win = 1
        game2_win = 1
        game3_win = "-"
      
      if check1.group() == "Retired":
        walkover = "FALSE"
        retired = "TRUE"
        game1_score = doc.find_all("div", {"class": re.compile("^game-score-1")})[0].get_text().replace("\n", "").replace(" ", "") + "-" + doc.find_all("div", {"class": re.compile("^game-score-2")})[0].get_text().replace("\n", "").replace(" ", "")
        game2_score = "-"
        game3_score = "-"
        
        if doc.find_all("div", {"class": re.compile("^game-score-1")})[0].get_text().replace("\n", "").replace(" ", "") > doc.find_all("div", {"class": re.compile("^game-score-2")})[0].get_text().replace("\n", "").replace(" ", ""):
          game1_win = 1
        else:
          game1_win = 2 
          game2_win = game1_win
          game3_win = "-"
    
    # H2H TAB
    url = "https://bwfworldchampionships.bwfbadminton.com/results/3996/totalenergies-bwf-world-championships-2021/" + date + "/" + "?match=" + str(number) + "&stab=h2h"
    result = requests.get(url)
    doc = BeautifulSoup(result.text, "html.parser")
    h2h = doc.find("div", {"class": "cifr1"}).get_text().replace("\n", "").replace(" ", "") + "-" + doc.find("div", {"class": "cifr2"}).get_text().replace("\n", "").replace(" ", "")

    # RANKING TAB
    url = "https://bwfworldchampionships.bwfbadminton.com/results/3996/totalenergies-bwf-world-championships-2021/" + date + "/" + "?match=" + str(number) + "&stab=ranking"
    result = requests.get(url)
    doc = BeautifulSoup(result.text, "html.parser")
    crank = doc.find("div", {"class": "cifr1"}).get_text().replace("\t", "").replace("\n", "").replace(" ", "") + "-" + doc.find("div", {"class": "cifr2"}).get_text().replace("\t", "").replace("\n", "").replace(" ", "")
    
    # GAME SCORE
    if game1_score == "-":
      score1_list = "-"
    else:
      url = "https://bwfworldchampionships.bwfbadminton.com/results/3996/totalenergies-bwf-world-championships-2021/"+ date + "/" + "?match=" + str(number) + "&stab=game&g=1"
      result = requests.get(url)
      doc = BeautifulSoup(result.text, "html.parser")
      body = doc.body
      st = body.find("script").string
      pattern = re.search(r'chartData = (.*?);.*?', st)
      score1_list = list()
      if pattern is not None:
        data = json.loads(pattern.groups()[0])
        for i in range(0,len(data)):
          score1_list.append(str(str(data[i].get("team1")) + "-" + str(data[i].get("team2"))))

    if game2_score == "-":
      score2_list = "-"
    else:
      url = "https://bwfworldchampionships.bwfbadminton.com/results/3996/totalenergies-bwf-world-championships-2021/"+ date + "/" + "?match=" + str(number) + "&stab=game&g=2"
      result = requests.get(url)
      doc = BeautifulSoup(result.text, "html.parser")
      body = doc.body
      st = body.find("script").string
      pattern = re.search(r'chartData = (.*?);.*?', st)
      score2_list = list()
      if pattern is not None:
        data = json.loads(pattern.groups()[0])
        for i in range(0,len(data)):
          score2_list.append(str(str(data[i].get("team1")) + "-" + str(data[i].get("team2"))))
      
    if game3_score == "-":
      score3_list = "-"    
    else:
      url = "https://bwfworldchampionships.bwfbadminton.com/results/3996/totalenergies-bwf-world-championships-2021/"+ date + "/" + "?match=" + str(number) + "&stab=game&g=3"
      result = requests.get(url)
      doc = BeautifulSoup(result.text, "html.parser")
      body = doc.body
      st = body.find("script").string
      pattern = re.search(r'chartData = (.*?);.*?', st)
      score3_list = list()
      if pattern is not None:
        data = json.loads(pattern.groups()[0])
        for i in range(0,len(data)):
          score3_list.append(str(str(data[i].get("team1")) + "-" + str(data[i].get("team2"))))

    row = [player1_team1_name, player2_team1_name, player1_team2_name, player2_team2_name, team1_flag, team2_flag, game1_score, game2_score, game3_score, game1_win, game2_win, game3_win, round, walkover, retired, h2h, crank, score1_list, score2_list, score3_list]
    rows.append(row)

In [ ]:
df = pd.DataFrame(rows, columns=["player1_team1_name", "player2_team1_name", "player1_team2_name", 'player2_team2_name', 'team1_flag', 'team2_flag', 'game1_score', 'game2_score', 'game3_score', 'game1_win', 'game2_win', 'game3_win', 'round', 'walkover', 'retired', 'h2h', 'crank', 'score1_list', 'score2_list', 'score3_list'])
df.shape

(20, 20)

In [ ]:
df

,player1_team1_name,player2_team1_name,player1_team2_name,player2_team2_name,team1_flag,team2_flag,game1_score,game2_score,game3_score,game1_win,game2_win,game3_win,round,walkover,retired,h2h,crank,score1_list,score2_list,score3_list
0,TAI Tzu Ying,-,PUSARLA V. Sindhu,-,TPE,IND,21-17,21-13,,1,1,,WS,FALSE,FALSE,17-5,2-7,"[0-0, 0-1, 1-1, 2-1, 2-2, 3-2, 4-2, 5-2, 6-2, ...","[0-0, 1-0, 1-1, 1-2, 2-2, 2-3, 3-3, 4-3, 4-4, ...",[]
1,KIDAMBI Srikanth,-,Mark CALJOUW,-,IND,NED,21-8,21-7,,2,2,,MS,FALSE,FALSE,1-0,13-28,"[0-0, 0-1, 1-1, 2-1, 3-1, 4-1, 5-1, 5-2, 6-2, ...","[0-0, 1-0, 1-1, 2-1, 3-1, 3-2, 4-2, 4-3, 5-3, ...",[]
2,Dechapol PUAVARANUKROH,Sapsiree TAERATTANACHAI,GOH Soon Huat,LAI Shevon Jemie,THA,MAS,21-15,21-13,,1,1,,XD,FALSE,FALSE,10-7,1-14,"[0-0, 1-0, 1-1, 1-2, 2-2, 2-3, 2-4, 3-4, 3-5, ...","[0-0, 1-0, 2-0, 2-1, 2-2, 3-2, 4-2, 5-2, 5-3, ...",[]
3,CHEN Qing Chen,JIA Yi Fan,Nami MATSUYAMA,Chiharu SHIDA,CHN,JPN,21-13,21-17,,1,1,,WD,FALSE,FALSE,5-3,1-5,"[0-0, 1-0, 2-0, 2-1, 2-2, 2-3, 3-3, 4-3, 5-3, ...","[0-0, 0-1, 0-2, 1-2, 1-3, 1-4, 2-4, 3-4, 3-5, ...",[]
4,Akane YAMAGUCHI,-,AN Se Young,-,JPN,KOR,21-7,11-21,21-10,2,2,1,WS,FALSE,FALSE,8-5,1-3,"[0-0, 0-1, 1-1, 2-1, 2-2, 3-2, 4-2, 5-2, 5-3, ...","[0-0, 0-1, 1-1, 1-2, 1-3, 1-4, 2-4, 2-5, 3-5, ...","[0-0, 1-0, 1-1, 1-2, 2-2, 2-3, 3-3, 4-3, 4-4, ..."
5,ONG Yew Sin,TEO Ee Yi,LEE Yang,WANG Chi-Lin,MAS,TPE,12-21,21-17,21-15,2,1,1,MD,FALSE,FALSE,2-3,10-4,"[0-0, 0-1, 1-1, 2-1, 3-1, 3-2, 4-2, 4-3, 4-4, ...","[0-0, 1-0, 2-0, 2-1, 3-1, 3-2, 4-2, 5-2, 6-2, ...","[0-0, 1-0, 1-1, 2-1, 3-1, 4-1, 5-1, 6-1, 7-1, ..."
6,Anders ANTONSEN,-,LEE Zii Jia,-,DEN,MAS,12-21,-,-,2,2,-,MS,FALSE,TRUE,3-1,3-5,"[0-0, 1-0, 1-1, 1-2, 1-3, 2-3, 2-4, 3-4, 3-5, ...",-,-
7,LOH Kean Yew,-,PRANNOY H. S.,-,SGP,IND,21-14,21-12,,1,1,,MS,FALSE,FALSE,1-2,8-18,"[0-0, 1-0, 2-0, 3-0, 3-1, 3-2, 3-3, 3-4, 4-4, ...","[0-0, 0-1, 0-2, 1-2, 2-2, 2-3, 3-3, 3-4, 4-4, ...",[]
8,KIM So Yeong,KONG Hee Yong,Rin IWANAGA,Kie NAKANISHI,KOR,JPN,21-8,21-18,,2,1,,WD,FALSE,FALSE,1-0,4-21,"[0-0, 0-1, 0-2, 0-3, 1-3, 2-3, 2-4, 3-4, 4-4, ...","[0-0, 1-0, 1-1, 2-1, 2-2, 2-3, 2-4, 3-4, 4-4, ...",[]
9,HE Ji Ting,TAN Qiang,GOH V Shem,TAN Wee Kiong,CHN,MAS,22-20,21-14,,1,1,,MD,FALSE,FALSE,1-1,19-15,"[0-0, 1-0, 2-0, 3-0, 4-0, 5-0, 5-1, 5-2, 6-2, ...","[0-0, 0-1, 1-1, 2-1, 3-1, 3-2, 3-3, 3-4, 4-4, ...",[]


In [ ]:
df.to_excel('df.xls', index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  """Entry point for launching an IPython kernel.
